# Notebook to test functions while the communication is not working

Libraries:
- pycryptodome : pip install pycryptodome
- cryptography : pip install cryptography
- pmlb: pip install pmlb
- pandas : pip install pandas

In [1]:
!pip uninstall pycrypto -y
!pip3 install pycryptodome pmlb

  Using cached pycryptodome-3.15.0-cp35-abi3-win_amd64.whl (1.9 MB)
  Using cached pmlb-1.0.1.post3-py3-none-any.whl (19 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached pandas-1.5.0-cp310-cp310-win_amd64.whl (10.4 MB)
  Using cached pytz-2022.4-py2.py3-none-any.whl (500 kB)
  Using cached numpy-1.23.3-cp310-cp310-win_amd64.whl (14.6 MB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)


In [3]:
import pandas as pd
from io import StringIO
from pmlb import fetch_data

from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os

In [15]:

file_path = os.path.join('dataset', 'infringement_dataset.csv')

In [16]:

pd.read_csv(file_path).head(3)

,Unnamed: 0,Table,Row,Description,Special
0,1,application_data,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_data,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_data,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN


#### Generate Private Key

In [17]:
def get_private_key():
    return get_random_bytes(AES.key_size[0])

    """
    with open('/dev/urandom', 'rb') as f:
        return f.read(AES.key_size[0])
    """

### AES-CTR

##### Encrypt

In [18]:
def encrypt_csv_ctr(file_name, civ, counter_size=128):
    key = get_private_key()
    
    counter = Counter.new(counter_size, initial_value=civ)
    
    aes = AES.new(key, mode=AES.MODE_CTR, counter=counter)

    with open(file_name, 'rb') as file:
        csv_data = file.read()
        encrypted_csv = aes.encrypt(csv_data)
    return encrypted_csv, key

ctr_iv = 10
ctr_ct, ctr_key = encrypt_csv_ctr(file_path, ctr_iv)

#### Decrypt

In [19]:
def decrypt_csv_ctr(ct, key, counter_iv, counter_size=128):
    counter = Counter.new(counter_size, initial_value=counter_iv)    
    aes = AES.new(key=key, mode=AES.MODE_CTR, counter=counter)
    return aes.decrypt(ct)

In [20]:
pd.read_csv(StringIO(decrypt_csv_ctr(ctr_ct, ctr_key, ctr_iv).decode('utf-8'))).head(3)

,Unnamed: 0,Table,Row,Description,Special
0,1,application_data,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_data,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_data,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
